Import packages and set paths

In [35]:
import os
import sys
# import emc2
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import matplotlib.colors as mcolors
# import matplotlib as mpl
import pandas as pd
import numpy as np
import xarray as xr
# import cftime
# from matplotlib.colors import LinearSegmentedColormap
# from matplotlib.ticker import MaxNLocator

# Navigate to my EMC2 project data directory
os.chdir('/g/data/jk72/ck4840/projects/emc2/data/')
# Check current project data directory
print("Project data directory:", os.getcwd())
# Create list of merged product files
lidar_radar_merged_list = os.listdir('marcus_merged_lidar_radar')

Project data directory: /g/data/jk72/ck4840/projects/emc2/data


UM regional model focus week dates:
2018-02-01 through 2018-02-07 (inclusive)

In [36]:
# Set the height values from the UM regional model to interpolate the lidar-radar data onto
# NB: REPLACE WITH ROUNDED Z0_HYBRID_HEIGHT VALUES FROM THE UM_REG DATA
z0_hybrid_height_rounded = [5,17,36,63,96,137,185,240,302,371,448,531,622,720,825,937,1056,1183,1316,1457,1605,1760,1922,2091,2268,2451,2642,2840,3045,3257,3476,3703,3936,4177,4425,4680,4942,5211,5488,5771,6063,6363,6672,6990,7318,7656,8005,8364,8734,9115,9509,9914,10333,10764,11209,11668,12142,12630,13134,13654,14191,14745,15317,15907,16516,17144,17793,18463,19154,19867,20604,21364,22148,22957,23793,24655,25544,26462,27409,28386,29394,30433,31505,32610,33750,34926,36137,37386,38673,40000]
# print(um_reg_heights)
print(len(um_reg_heights))

90


In [37]:
# Load the merged lidar-radar product
current_file = lidar_radar_merged_list[0]
lidar_radar_merged = xr.open_dataset(f'marcus_merged_lidar_radar/{current_file}', decode_times = False)

# Convert the 'height' variable to metres
lidar_radar_merged['height'] = lidar_radar_merged['height'] * 1000

lidar_radar_merged

<xarray.Dataset>
Dimensions:               (hour: 1440, height: 1001, status: 3)
Coordinates:
  * hour                  (hour) float32 0.0 0.01667 0.03333 ... 23.97 23.98
  * height                (height) float32 7.495 22.48 ... 1.498e+04 1.5e+04
    status                (hour) float32 ...
Data variables: (12/14)
    Fog_Or_Low_Cloud      (hour) float32 ...
    rcldmask              (height, hour) float32 ...
    lcldmask              (height, hour) float32 ...
    cld_mask              (height, hour) float32 ...
    cld_phase             (height, hour) float32 ...
    copol_backscatter     (height, hour) float32 ...
    ...                    ...
    T                     (height, hour) float32 ...
    P                     (height, hour) float32 ...
    reflectivity          (height, hour) float32 ...
    doppler               (height, hour) float32 ...
    cld_nlayer            (status, hour) float64 ...
    cld_cover_type        (status, height, hour) float32 ...

In [38]:
# Add a 'time' variable to the lidar-radar data rounded to the nearest X1 minutes to match the UM_reg data timestamps

# Extract the current date string
current_date = current_file.split('.')[1]

# Convert the 'hour' variable to HH:MM:SS datetime format
hour_values = lidar_radar_merged['hour'].values
hours_in_seconds = hour_values * 3600
time_format = '%H:%M:%S'
hours_as_time = pd.to_datetime(hours_in_seconds, unit='s').strftime(time_format)

# Combine 'current_date' with 'hour' to create 'time' variable
combined_time = pd.to_datetime(current_date, format='%Y%m%d') + pd.to_timedelta(hours_as_time)

# Round time_1min to the nearest 10 minutes with a -1 offset, i.e. to the nearest X1 minutes
time_1min = combined_time
time_1min_offset = time_1min - pd.Timedelta(minutes=1)
time_1min_offset_rounded = time_1min_offset.round('10min')
time_1min_rounded = time_1min_offset_rounded + pd.Timedelta(minutes=1)
time_data_array = xr.DataArray(time_1min_rounded, dims=('time',), coords={'time': time_1min_rounded})
lidar_radar_merged = lidar_radar_merged.assign_coords(time=time_data_array)

# Compare the number of unique timestamps in the original versus the updated time
# The ratio should be approximately 10:1
print(len(np.unique(lidar_radar_merged.hour.values)))
print(len(np.unique(lidar_radar_merged.time.values)))

1440
145


In [39]:
# Examine the dataset with the added 'time' variable
# Drop the old 'hour' variable
lidar_radar_merged = lidar_radar_merged.drop_vars('hour')
lidar_radar_merged

<xarray.Dataset>
Dimensions:               (height: 1001, hour: 1440, status: 3, time: 1440)
Coordinates:
  * height                (height) float32 7.495 22.48 ... 1.498e+04 1.5e+04
    status                (hour) float32 ...
  * time                  (time) datetime64[ns] 2018-02-05T00:01:00 ... 2018-...
Dimensions without coordinates: hour
Data variables: (12/14)
    Fog_Or_Low_Cloud      (hour) float32 ...
    rcldmask              (height, hour) float32 ...
    lcldmask              (height, hour) float32 ...
    cld_mask              (height, hour) float32 ...
    cld_phase             (height, hour) float32 ...
    copol_backscatter     (height, hour) float32 ...
    ...                    ...
    T                     (height, hour) float32 ...
    P                     (height, hour) float32 ...
    reflectivity          (height, hour) float32 ...
    doppler               (height, hour) float32 ...
    cld_nlayer            (status, hour) float64 ...
    cld_cover_type        (status, height, hour) float32 ...

In [66]:
print(z0_hybrid_height_rounded)
print(height_rounded)

[5, 17, 36, 63, 96, 137, 185, 240, 302, 371, 448, 531, 622, 720, 825, 937, 1056, 1183, 1316, 1457, 1605, 1760, 1922, 2091, 2268, 2451, 2642, 2840, 3045, 3257, 3476, 3703, 3936, 4177, 4425, 4680, 4942, 5211, 5488, 5771, 6063, 6363, 6672, 6990, 7318, 7656, 8005, 8364, 8734, 9115, 9509, 9914, 10333, 10764, 11209, 11668, 12142, 12630, 13134, 13654, 14191, 14745, 15317, 15907, 16516, 17144, 17793, 18463, 19154, 19867, 20604, 21364, 22148, 22957, 23793, 24655, 25544, 26462, 27409, 28386, 29394, 30433, 31505, 32610, 33750, 34926, 36137, 37386, 38673, 40000]
<xarray.DataArray 'height' (height: 1001)>
array([    7,    22,    37, ..., 14967, 14982, 14997])
Coordinates:
  * height   (height) float32 7.495 22.48 37.47 ... 1.497e+04 1.498e+04 1.5e+04


In [67]:
# Add a 'z0_hybrid_height' variable to the lidar-radar data rounded to the nearest z0_hybrid_height_rounded value to match the UM_reg data resolution

# Convert z0_hybrid_height_rounded to a numpy array
z0_hybrid_height_rounded_nparray = np.array(z0_hybrid_height_rounded)

# Initialize an empty array to store the rounded heights
rounded_heights = np.zeros_like(height_rounded)

# Loop through each value in height_rounded and find the nearest value in z0_hybrid_height_rounded
for i, h in enumerate(height_rounded):
    nearest_index = np.abs(z0_hybrid_height_rounded - h).argmin()
    rounded_heights[i] = z0_hybrid_height_rounded_nparray[nearest_index]



    
# Round 'height' to the nearest value in um_reg_heights
# height_rounded = np.round(lidar_radar_merged['height']).astype(int)
# height_rounded = np.atleast_1d(height_rounded)
# nearest_index = np.abs(z0_hybrid_height_rounded - height_rounded[:, np.newaxis]).argmin(axis=1)
# height_rounded = z0_hybrid_height_rounded[nearest_index]

# # Create a new variable 'z0_hybrid_height' with the rounded heights
# # lidar_radar_merged['z0_hybrid_height'] = xr.DataArray(height_rounded, dims=('time',), coords={'time': lidar_radar_merged['time']})
# lidar_radar_merged['z0_hybrid_height'] = xr.DataArray(height_rounded, dims=('height',), coords={'height': lidar_radar_merged['height']})

ValueError: dimensions () must have the same length as the number of data dimensions, ndim=1

In [50]:
lidar_radar_merged.z0_hybrid_height.values

array([    7,    22,    37, ..., 14967, 14982, 14997])

In [51]:
height_rounded

<xarray.DataArray 'height' (height: 1001)>
array([    7,    22,    37, ..., 14967, 14982, 14997])
Coordinates:
  * height   (height) float32 7.495 22.48 37.47 ... 1.497e+04 1.498e+04 1.5e+04